In [1]:
import openai, load_api_key
from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, ServiceContext

# 将整个./data/mr_fujino 的目录给加载进来。这里面的每一个文件，都会被当成是一篇文档。
documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
# 将所有的文档交给了 GPTSimpleVectorIndex 构建索引，它会把文档分段转换成一个个向量，然后存储成一个索引。
index = GPTSimpleVectorIndex.from_documents(documents)

index.save_to_disk('index_mr_fujino.json')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 6763 tokens


In [5]:
# 把刚才生成的索引加载到内存里面来。
index = GPTSimpleVectorIndex.load_from_disk('index_mr_fujino.json')
#  实际会调用openAI，但会通过 Embedding 相似度找出来的内容填入到问题中。
response = index.query("鲁迅先生在日本学习医学的老师是谁？")
print(response)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2984 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 34 tokens



鲁迅先生在日本学习医学的老师是藤野严九郎先生。


In [6]:
response = index.query("鲁迅先生去哪里学的医学？", verbose=True)
print(response)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2968 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 25 tokens



鲁迅先生去仙台的医学专门学校学习医学。


### 实际通过上下文让AI 来回答
1. 这个模版的开头，我们告诉 AI，我们为 AI 提供了一些上下文信息（Context information）。
2. 模版里面支持两个变量，一个叫做 context_str，另一个叫做 query_str。context_str 的地方，在实际调用的时候，会被通过 Embedding 相似度找出来的内容填入。而 query_str 则是会被我们实际提的问题替换掉。
3. 实际提问的时候，我们告诉 AI，只考虑上下文信息，而不要根据自己已经有的先验知识（prior knowledge）来回答问题。

In [7]:
from llama_index import QuestionAnswerPrompt

query_str = "鲁迅先生去哪里学的医学？"
DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)
QA_PROMPT = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

response = index.query(query_str, text_qa_template=QA_PROMPT, verbose=True)
print(response)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2968 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 25 tokens



鲁迅先生去仙台的医学专门学校学习医学。


## 通过llama_index对于文章进行小结

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import SpacyTextSplitter
from llama_index import GPTListIndex, LLMPredictor, ServiceContext
from llama_index.node_parser import SimpleNodeParser

# define LLM
# 设置模型输出的内容都在 1024 个 Token 以内，这样可以确保我们的小结不会太长，不会把一大段不相关的内容都合并到一起去。
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=1024))

# 进行中文文本的分割，这里用了中文的语言模型zh_core_web_sm
# 限制了分割出来的文本段，最长不要超过 2048 个 Token
text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=2048)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
nodes = parser.get_nodes_from_documents(documents)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
# 建立索引的时候，并不会创建 Embedding，所以索引创建的时候很快，也不消耗 Token 数量
list_index = GPTListIndex(nodes=nodes, service_context=service_context)

response = list_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", response_mode="tree_summarize")
print(response)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:llama_index.indices.common_tree.base:> Building index from nodes: 2 chunks
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 10031 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


鲁迅先生在日本仙台的医学专门学校学习时，遇到了藤野严九郎教授。藤野教授很有学问，对学生也很关心，甚至帮助鲁迅修改讲义。但鲁迅当时不够用功，有时也很任性。最终，他决定离开医学，去学习生物学。在他离开仙台之前，藤野先生给了他一张照片，并希望他能够保持联系。鲁迅先生很久没有和一个人通信了，尽管他想写信，但是他不知道该写些什么。他想起了一个人，这个人是他的老师，他对鲁迅很热心，给他很多鼓励和教诲。这个老师希望中国有新的医学，他的性格很伟大，但是他的名字并不为人所知。鲁迅曾经收藏了他的讲义，但是在搬家的时候丢失了。他的照片还挂在鲁迅的房间里，每当鲁迅疲倦的时候，看到他的照片就会感到勇气和良心发现。
